In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
import math

## Group Assignment
### Team Number: 4
### Team Member Names: Jacky Xu, Esha Kumar, Jingyi Fang
### Team Strategy Chosen: SAFE

In [2]:
# read the ticker file
tickers = pd.read_csv("Tickers.csv",header=None)
# rename the column as "ticker"
tickers.columns=['ticker']
# create a list that contains all the tickers
ticker_lst = tickers['ticker'].tolist()



In [3]:

# call_tickers takes a list of tickers and return a list of called tickers
def call_tickers(lst):
    # create an empty list to store the called tickers
    called_ticker_lst = []
    # iterate the ticker list
    for i in lst:
        # call each ticker and append it to the called ticker list
        called_ticker_lst.append(yf.Ticker(i))
    # return the called ticker list when finish
    return called_ticker_lst

# get_ticker_info takes a list of called tickers and returns the currency, current prices and symbols in a dataframe,
# as well as a called ticker list with all the delisted tickers removed
def get_ticker_info(lst):
    # create an empty dataframe and an empty list
    ticker_info = pd.DataFrame()
    called_tickers = []
    # iterate the called ticker list
    for i in lst:
        # extract the ticker names(since i is yfinance.Ticker object <ticker_name>)
        ticker_name = str(i)[24:-1]
        # use try and except to skip the delisted tickers
        try:
            # extract the info we need and store into the dataframe
            ticker_info[ticker_name] = [i.info['currency'], i.info['currentPrice'], i.info['symbol']]
            # also append the ticker to the called tickers list
            called_tickers.append(i)
        except:
            pass
    # set the index for the dataframe
    ticker_info.index = ['currency','current_price','symbol']
    # return the dataframe and the called tickers list
    return ticker_info,called_tickers


# filter_daily_volume takes the ticker info dataframe, the called ticker list, start date, end date, 
# returns a list of tickers with the required average daily volume and the updated ticker info dataframe
def filter_daily_volume(df,lst,start,end):
    # write a for loop to iterate the called ticker list
    for i in lst:
        # extract the volume of the stock in the given time period 
        avg_daily_volumn = i.history(start = start, end = end,interval='1d')['Volume'].mean()
        # check if the average daily volume is < 10000
        if avg_daily_volumn < 10000:
            # if yes, remove the ticker from the list and drop the ticker from the ticker info dataframe
            lst.remove(i)
            df.drop(columns = str(i)[24:-1])
        # return the list and the dataframe
    return lst,df


# filter_USD takes a list of called tickers and the ticker info dataframe and delete all the non-USD tickers from both
def filter_USD(lst,df):
    # Transpose the dataframe so that the ticker name becomes the index and the information becomes columns
    df = df.T
    # filter out the USD tickers in the dataframe
    df = df[df.currency == 'USD']
    for i in lst:
        ticker_name = str(i)[24:-1]
        # iterate the called ticker list, remove the ticker that is not in the filtered dataframe
        if ticker_name not in df.index:
            lst.remove(i)
    # return the list and the dataframe
    return lst, df

# get_prices takes a list of called tickers, start date adn end date, returns the historical prices in the given time period
def get_prices(lst,start,end):
    prices = pd.DataFrame()
    for i in lst:
        prices[str(i)[24:-1]] = i.history(start = start,end=end,interval = '1d')['Close']
    return prices

# how about daily returns? since the competition only last for 6 days
# get_monthly_returns takes the historical prices(or portfolio value) and returns the monthly returns
def get_monthly_returns(prices):
    monthly_returns = prices.resample('MS').first().pct_change()
    monthly_returns = monthly_returns.iloc[1:]
    return monthly_returns
    
# get_std takes the prices of the stock and returns the std of the prices
def get_std(prices):
    std = pd.DataFrame(columns = ('ticker','std'))
    index = 0
    for i in prices.columns:
        std.loc[index] = [i, prices[i].std()]
        index += 1
    return std
# rank takes a dataframe, column name and method and rank the dataframe by the given column using the given method
def rank(df,column_name,method):
    # use the build-in rank function to rank the dataframe, store the rankings in a new column
    df[column_name+'_rank'] = df[column_name].rank(method=method)
    # return the ranked dataframe
    return df

# get_std_beta_rank takes the ranked std dataframe and the ranked beta dataframe, rank the dataframe by both and return the top 40 stocks
def get_std_beta_rank(std,beta):
    # combine the std dataframe and the beta dataframe
    rank = pd.concat([std,beta['beta'],beta['beta_rank']],join='inner',axis=1)
    # add up the ranks to get the final rank
    rank['final_rank'] = rank['std_rank'] + rank['beta_rank']
    # rank by the final rank
    rank = rank.sort_values(by='final_rank')
    # takes the top 40 stocks
    rank = rank[:40]
    # return the dataframe
    return rank

In [4]:
# set up the start date and the end date
start_date = "2021-07-02"
end_date = "2021-10-22"

# call the tickers
called_tickers_lst = call_tickers(ticker_lst)

# collect the info in info dictionary at once
ticker_info,called_tickers = get_ticker_info(called_tickers_lst)


In [5]:
# filter the tickers with required average daily volumn_2 = ticker_info
daily_volumn_tickers, daily_volumn_tickers_df = filter_daily_volume(ticker_info,called_tickers, start_date, end_date)

# filter the USD tickers
good_tickers, good_tickers_df = filter_USD(daily_volumn_tickers, daily_volumn_tickers_df)

# extract the closing prices for the good tickers
prices = get_prices(good_tickers, start_date, end_date)

# Get a list of Ticker Symbols (list of Strings)
ticker_symbols = prices.columns

# calculate the std according to the closing prices
std = get_std(prices)

# rank them in ascending order
std = rank(std,'std','min')

In [39]:
good_tickers

[yfinance.Ticker object <AAPL>,
 yfinance.Ticker object <ABBV>,
 yfinance.Ticker object <ABT>,
 yfinance.Ticker object <ACN>,
 yfinance.Ticker object <AIG>,
 yfinance.Ticker object <AMZN>,
 yfinance.Ticker object <AXP>,
 yfinance.Ticker object <BA>,
 yfinance.Ticker object <BAC>,
 yfinance.Ticker object <BIIB>,
 yfinance.Ticker object <BK>,
 yfinance.Ticker object <BLK>,
 yfinance.Ticker object <BMY>,
 yfinance.Ticker object <C>,
 yfinance.Ticker object <CAT>,
 yfinance.Ticker object <CL>,
 yfinance.Ticker object <CMCSA>,
 yfinance.Ticker object <COF>,
 yfinance.Ticker object <COP>,
 yfinance.Ticker object <COST>,
 yfinance.Ticker object <CSCO>,
 yfinance.Ticker object <CVS>,
 yfinance.Ticker object <GM>,
 yfinance.Ticker object <GOOG>,
 yfinance.Ticker object <JPM>,
 yfinance.Ticker object <KMI>,
 yfinance.Ticker object <KO>,
 yfinance.Ticker object <LLY>,
 yfinance.Ticker object <LMT>,
 yfinance.Ticker object <MO>,
 yfinance.Ticker object <MON>,
 yfinance.Ticker object <MRK>,
 yfinan

In [6]:
# Getting the Symbol (Ticker) of each Stock
def get_ticker_Symbol (lst):
    ticker_lst = []
    for ticker in lst:
        ticker_lst.append(ticker.info['symbol'])
    return ticker_lst

In [7]:
# Calculating Beta
def get_beta(good_tickers, prices, start_date, end_date, ticker_symbols):
    Ticker = '^GSPC'
    MarketIndex = yf.Ticker(Ticker) # The symbol yfinance uses for the S&P 500

    MarketIndex_hist = MarketIndex.history(start=start_date, end=end_date)

    # DataFrame for Market Index
    marketDF = pd.DataFrame(MarketIndex_hist['Close'])
    marketDF.columns = [Ticker]
    

    # Loop iterates through the column of prices
    betaList = []
    for i in range(len(ticker_symbols)):
        currentStockPrice = prices[ticker_symbols[i]]
        currentStockPrice = pd.concat([currentStockPrice, marketDF], join = 'inner', axis=1)
        
        # Getting the Monthly Return of each stock
        monthly_returns = currentStockPrice.resample('MS').first().pct_change()  # Dropping the first entry (since it's N/A)
        monthly_returns.drop(index=monthly_returns.index[0], inplace=True)
        
        
        # Calculate the market variance (you will need to reference the column correponding to the market)
        MarketVar = monthly_returns[Ticker].var()
        
        betaList.append(monthly_returns.cov() / MarketVar)
        
    # Filter through betaList and extract the beta for each stock
    for i in range(len(betaList)):
        betaList[i] = betaList[i].iat[0,1]
    
    # Creating a DataFrame for Tickers and their Beta value
    beta = pd.DataFrame(columns = ('ticker','beta'))
    for i in range(len(good_tickers)):
        beta.loc[i] = [str(good_tickers[i])[24:-1], betaList[i]]

    return beta

In [8]:
# calculate the beta according to the closing prices
beta = get_beta(good_tickers, prices, start_date, end_date, ticker_symbols)

# rank them in ascending order
beta = rank(beta,'beta','min')

# combine the rank of std and beta, then rank the combined rank
std_beta_rank = get_std_beta_rank(std,beta)
std_beta_rank.head()

,ticker,std,std_rank,beta,beta_rank,final_rank
25,KMI,0.760351,3.0,-1.738541,5.0,8.0
48,T,0.539296,2.0,-0.460408,11.0,13.0
30,MON,0.046874,1.0,-0.146849,13.0,14.0
45,SLB,2.118426,10.0,-1.906580,4.0,14.0
54,USB,2.333448,12.0,-1.302068,7.0,19.0


In [9]:
# pair_by_corr takes the ranked dataframe and the prices of the stocks, 
# returns 10 portfoliois in one dataframe, each with a pair of stocks with the lowest correlation
def pair_by_corr(rank, prices):
    # open an empty dataframe
    pairs = pd.DataFrame()
    # counter is the number of pairs
    counter = 0
    # loop until we have 10 pairs
    while counter < 10:
        # min_corr stores the lowest correlation
        min_corr = 1
        # self is the stock that has lowest std and beta(which is the safest one)
        self = rank.iloc[0,0]
        # get the monthly_returns for self
        self_monthly_returns = pd.DataFrame(get_monthly_returns(prices[self]))
        # for all other stocks in the ranked dataframe
        for i in range (1,len(rank)-1):
            # candidate is the stock that has ith lowest std and beta
            candidate = rank.iloc[i,0]
            # get the monthly_returns for the candidate
            candidate_monthly_returns = pd.DataFrame(get_monthly_returns(prices[candidate]))
            # calculate the correlation betwenn self and candidate
            corr = self_monthly_returns[self].corr(candidate_monthly_returns[candidate])
            # when the correlation is less than the lowest correlation
            if corr <= min_corr:
                # replace the min_corr by the current correlation
                min_corr = corr
                # candidate becomes the potential champion that has the lowest correlation with self
                champion = candidate
                champion_monthly_returns = candidate_monthly_returns
        # get current price from the ticker info dataframe
        self_current_price = ticker_info.loc['current_price',self]
        champion_current_price = ticker_info.loc['current_price',champion]
        # calculate the number of shares we could purchase using the current price
        self_shares = 500000 /  self_current_price
        champion_shares = 500000 /  champion_current_price
        # get monthly price, portfolio value, and portfolio return for self and champion
        pairs[self+'_price'] = prices[self].resample('MS').first()
        pairs[self+'_value'] =  pairs[self+'_price']*self_shares
        pairs[self+'_return'] = pd.DataFrame(get_monthly_returns(pairs[self+'_value']))
        pairs[champion+'_price'] = prices[champion].resample('MS').first()
        pairs[champion+'_value'] =  pairs[champion+'_price']*champion_shares
        pairs[champion+'_return'] = pd.DataFrame(get_monthly_returns(pairs[champion+'_value']))
        # remove self and champion from the ranked dataframe to prepare for the next iteration
        rank = rank[1:]
        rank = rank[rank['ticker'] != str(champion)]
        # add 1 to the counter for each finished pair
        counter += 1
    # return the pairs
    return pairs
            
        
        

In [10]:
# get the portfolio of pairs 
pairs = pair_by_corr(std_beta_rank,prices)
pairs.head()

,KMI_price,KMI_value,KMI_return,AAPL_price,AAPL_value,AAPL_return,T_price,T_value,T_return,MO_price,...,BAC_return,ABBV_price,ABBV_value,ABBV_return,CVS_price,CVS_value,CVS_return,AXP_price,AXP_value,AXP_return
Date,,,,,,,,,,,,,,,,,,,,,
2021-07-01,18.005268,552987.349414,NaN,139.960007,434604.417817,NaN,27.617552,566165.473628,NaN,46.675827,...,NaN,112.535881,486536.450681,NaN,81.460457,427210.283449,NaN,168.082153,491755.861089,NaN
2021-08-01,17.050467,523662.977195,-0.053029,145.520004,451869.346269,0.039726,27.045464,554437.547397,-0.020715,46.872234,...,-0.077521,114.067108,493156.541955,0.013607,81.933922,429693.317673,0.005812,168.850250,494003.072686,0.004570
2021-09-01,15.908519,488590.871966,-0.066975,152.509995,473574.694159,0.048035,26.160303,536291.576791,-0.032729,49.180004,...,0.085090,110.925194,479572.822251,-0.027544,85.214851,446899.786970,0.040044,164.032227,479907.040850,-0.028534
2021-10-01,16.646847,511266.792729,0.046411,142.649994,442957.377644,-0.064652,26.131439,535699.860783,-0.001103,45.860001,...,0.051245,107.783287,465989.135531,-0.028325,83.584328,438348.687318,-0.019134,173.508667,507632.144506,0.057772


In [43]:
# get_sharpe_ratio takes a portfolio and return its sharpe ratio
def get_sharpe_ratio(df):
    # since the risk free rate is set to 0, then it is just the ratio between the expected return and the std of the returns
    return df['return'].mean()/df['return'].std()

# when 'stock' is True, combine_portfolio takes two stocks and the pairs dataframe, 
# returns a weight and the weighted portfolio with the lowest sharpe ratio
# when 'stock' is False, combine_portfolio takes two portfolio(old and new), the min and max range for the sharpe ratio,
# returns a weight and the weighted portfolio with the lowest sharpe ratio
def combine_portfolio(old,new,range_min,range_max,stock,stock1,stock2,pairs):
    # open an empty portfolio 
    candidate = pd.DataFrame()
    # min_ratio stores the lowest sharpe ratio
    min_ratio = 100
    # weight_1 is the weight of stock1 or the portfolio weight for old
    weight_1 = 0
    # iterates the given range
    for i in range(range_min,range_max+1):
        if stock:
            # for the stock version, the porfolio value is taken from the pairs dataframe
            # multiply the weight i with the value of each stock
            # add them up and double the result(since each stock value starts from 500000 and the required amount is 1 million)
            candidate['value'] = (pairs[stock1+'_value']*(i/100)+pairs[stock2+'_value']* (100-i/100))*2
        else:
            # for the portfolio version, just multiply the portfolio values by the weights and add them up
            candidate['value'] = old['value']*(i/100)+new['value']* (100-i/100)
        # get the monthly returns 
        candidate['return'] = pd.DataFrame(get_monthly_returns(candidate['value']))
        # get the sharpe ratio 
        sharpe_ratio = get_sharpe_ratio(candidate)
        # if the absolute value of the sharpe ratio(since the ratio could be negative too) is less than the min_ratio
        if abs(sharpe_ratio) <= min_ratio:
            # replace the min_ratio by the the absolute value of the current sharpe ratio 
            min_ratio = abs(sharpe_ratio)
            # update the champion and the weight
            champion = candidate
            weight_1 = i
    # return the optimal weight and the portfolio with the optimal weight
    return weight_1, champion

# get_range takes the optimal weight for stock1 and returns the range for the portfolio weight
def get_range(stock1_weight):
    if (stock1_weight == 0 or stock1_weight == 100):
        range_max = 1
        range_min = 0
    # if stock1 has a greater weight, the max portfolio weight would be 35/stock1_weight
    # since 35 is the max weight, and the weight of each stock is given by the product of stock1_weight * portfolio weight
    # so the max portfolio weight is 35/stock1_weight. Same logic for the min portfolio weight
    else:
        if stock1_weight >= 50:
            range_max = 35/stock1_weight
            range_min = 2.5/(100-stock1_weight)
            # if stock1 has a lower weight, just flip the stock1 and the other stock
        else:
            range_max = 35/(100-stock1_weight) 
            range_min = 2.5/stock1_weight
    print(range_min, range_max)
    return math.ceil(range_min*100),math.floor(range_max*100)


In [44]:
## Not sure if it's working

# open an empty list for the protfolio weights
portfolio_weights = []
# extract the column names for prices in the pairs dataframe( 'tickername_price')
paris_price_only = pairs.filter(like='_price', axis=1).columns
# get the ticker name by removing the '_price' for stock1 and stock2
stock1 = paris_price_only[0][:-6]
stock2 = paris_price_only[1][:-6]
# combine stock1 and stock2, get the optimal weight as w1 and the weighted portfolio as old
# we are initiating old, which is the existing portfolio
w1,old = combine_portfolio(0,0,0,100,True,stock1,stock2,pairs)
# remove stock1 and stock2
print(paris_price_only)
paris_price_only = paris_price_only[2:]
print(paris_price_only)
# then iterates the rest of the stocks
for i in range(9):
    # get the ticker names for the pair of stocks
    stock1 = paris_price_only[0][:-6]
    stock2 = paris_price_only[1][:-6]
    print(stock1)
    print(stock2)
    # combine the pair of stock, store the optimal stock weight as w1 and the weighted portfolio as new
    w1, new = combine_portfolio(0,0,0,100,True,stock1,stock2,pairs)
    print(w1)
    # get the range for portfolio weight with the optimal weight w1
    range_min,range_max = get_range(w1)
    print(range_min, range_max)
    # combine the portfolio new that we just created with the existing portfolio old using the range
    # store the optimal portfolio weight and the combined portfolio as champion
    portfolio_weight, champion = combine_portfolio(old,new,range_min,range_max,False,0,0,0)
    # append the protfolio weight into portfolio weights
    portfolio_weights.append(portfolio_weight)
    print(portfolio_weights)
    print(champion)
    
    # mutiply all the portfolio weights by the current portfolio weight
    # I feel like we should do this but I don't know the reason behind yet
    #portfolio_weights = [i * portfolio_weight for i in portfolio_weights]
    
    
    # the combined portfolio becomes the existing old portfolio for the next pair
    old = champion
    # remove the pair
    paris_price_only = paris_price_only[2:]
    print()


Index(['KMI_price', 'AAPL_price', 'T_price', 'MO_price', 'MON_price',
       'BK_price', 'SLB_price', 'KO_price', 'USB_price', 'BMY_price',
       'MRK_price', 'PFE_price', 'OXY_price', 'PEP_price', 'GM_price',
       'PM_price', 'BAC_price', 'ABBV_price', 'CVS_price', 'AXP_price'],
      dtype='object')
Index(['T_price', 'MO_price', 'MON_price', 'BK_price', 'SLB_price', 'KO_price',
       'USB_price', 'BMY_price', 'MRK_price', 'PFE_price', 'OXY_price',
       'PEP_price', 'GM_price', 'PM_price', 'BAC_price', 'ABBV_price',
       'CVS_price', 'AXP_price'],
      dtype='object')
T
MO
0
0 1
0 100
[100]
                   value    return
Date                              
2021-07-01  1.059394e+10       NaN
2021-08-01  1.063871e+10  0.004227
2021-09-01  1.115329e+10  0.048368
2021-10-01  1.040779e+10 -0.066841

MON
BK
0
0 1
0 100
[100, 100]
                   value    return
Date                              
2021-07-01  1.927669e+10       NaN
2021-08-01  1.938753e+10  0.005750
2021-09-01 

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.